<a href="https://colab.research.google.com/github/mingd00/menu-translation/blob/main/menu_mapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 라벨 데이터의 클래스, 메뉴 정보를 매핑하여 새로운 json 파일 생성

### 0. 라이브러리 설치

In [ ]:
import pandas as pd
import numpy as np
import os
import zipfile
import glob
import shutil
import json

### 1. 경로 설정

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = '/content/drive/MyDrive/menu_project'

### 2. 데이터 불러오기

In [ ]:
data = os.path.join(path + '/data/meta_data.zip')
data

'/content/drive/MyDrive/menu_project/data/meta_data.zip'

### 3. json 파일에서 필요한 정보를 추출하여 새로운 파일에 저장

* 새로운 파일, 변수 선언

In [ ]:
# 새로운 json 파일 생성
with open('menu_info.json', 'w') as f:
  json.dump({}, f, indent=4)

# 현재까지 매핑된 메뉴명을 저장할 리스트와 max_class_id 변수
output_data = []
max_class_id = -1

* 정보 추출 후 타겟 json 파일로 옮기는 함수

In [ ]:
def change_json(input_file, output_file):
  global max_class_id, output_data

  # 입력 파일 읽기
  data = json.load(input_file)

  for annotation in data.get("annotations", []):
    menu_info = annotation.get("menu_information", {})
    menu_name_ko = menu_info.get("ko")

    # 메뉴 이름 중복 확인
    if any(entry["ko"] == menu_name_ko for entry in output_data):
      continue # 존재하면 넘어감

    # 새 항목 생성 후 추가
    new_entry = {
        "class": max_class_id + 1,
        "ko": menu_info.get("ko"),
        "en.ROMAN": menu_info.get("en.ROMAN"),
        "en": menu_info.get("en"),
        "ja": menu_info.get("ja"),
        "price": menu_info.get("price")
    }

    output_data.append(new_entry)
    max_class_id += 1

    # 결과를 출력 파일로 저장
    with open(output_file, "w") as f:
      json.dump(output_data, f, indent=4, ensure_ascii=False)

* 압출 파일 돌며 모든 json 파일 변환하는 함수

In [ ]:
# zip 파일 풀어서 파일 옮기기
def move_json(target_folder, tmp_folder):
  for target in target_folder:
    zip_ref.extract(target, tmp_folder) # zip 파일을 임시 폴더로 추출
    extracted_zip_path = os.path.join(tmp_folder, target) # 추출된 zip 파일 경로

    # 내부 zip 파일 열기
    with zipfile.ZipFile(extracted_zip_path, 'r') as inner_zip:
      # 추출된 파일 변환
      for json_file in inner_zip.namelist():
        if json_file.endswith('.json'):
          input_file = inner_zip.open(json_file)
          change_json(input_file, "menu_info.json")
      print(f'{target} 추출 완료')

* train, valid 데이터 각각 처리

In [ ]:
# 임시 폴더 (압축 해제용)
tmp_folder = '/content/tmp'
os.makedirs(tmp_folder, exist_ok=True)

# 압축 파일 열기
with zipfile.ZipFile(data, 'r') as zip_ref:
  # 이미지 파일 선택
  train_image = [f for f in zip_ref.namelist() if 'labels/train/' in f and f.endswith('.zip')]
  valid_image = [f for f in zip_ref.namelist() if 'labels/val/' in f and f.endswith('.zip')]

  # 'train', 'valid' 라벨 파일들 출력
  print(f"Train labels: {train_image}, Valid labels: {valid_image}")

  move_json(train_image, tmp_folder)
  move_json(valid_image, tmp_folder)

Train labels: ['labels/train/busan.zip', 'labels/train/seoul.zip'], Valid labels: ['labels/val/jeolla.zip']
labels/train/busan.zip 추출 완료
labels/train/seoul.zip 추출 완료
labels/val/jeolla.zip 추출 완료


## 4. 파일 다운로드

In [ ]:
# JSON 파일을 Google Drive로 이동

shutil.move("menu_info.json", "/content/drive/My Drive/menu_info.json")
print("JSON 파일을 Google Drive에 저장 완료!")

JSON 파일을 Google Drive에 저장 완료!
